In [1]:
from os import getenv
from dotenv import load_dotenv
import graphviz 
# from relation_algorithms.relation_extraction_functions import LLM_Relation_Extractor
from src.extractor import relationExtractor
import os
import pandas as pd
from src.utils import normalize_text

load_dotenv()

dsa_2214_link = getenv('dsa_2214')
dsa_6114_link = getenv('dsa_6114')
hadoop_link = getenv('hadoop')
token = getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = token

/home/zanehutchens/uncc/research/rag/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute
<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type swigvarlink has no __module__ attribute


In [2]:
data = pd.read_csv('data/sorting.csv')
data.columns = ['concept', 'outcome']
data

,concept,outcome
0,Data Structures->Basics->Algorithm Analysis,Apply time complexity analysis guideline to an...
1,Data Structures->Basics->Algorithm Analysis->O...,Demonstrate an understanding of big O notation...
2,Algorithms->Non-recursive Algorthims,Demonstrate an understanding of non-recursive ...
3,Algorithms->Non-recursive Algorithms->Search,Apply the Comparable interface for object comp...
4,Algorithms->Non-recursive Algorithms->Search->...,Demonstrate an understanding of linear search;...
5,Algorithms->Non-recursive Algorithms->Search->...,Demonstrate an understanding of binary search;...
6,Algorithms->Non-recursive Algorithms->Sort,Demonstrate an understanding of sorting;
7,Algorithms->Non-recursive Algorithms->Sort->In...,Demonstrate an understanding of insertion sort...
8,Algorithms->Non-recursive Algorithms->Sort->Se...,Demonstrate an understanding of selection sort...
9,Algorithms->Non-recursive Algorithms->Sort->Bu...,Demonstrate an understanding of bubble sort;An...


In [3]:
concept_data = data['concept'].tolist()
actual_concepts = []
for string in concept_data:
    words = string.split('->')
    for word in words:
        if word not in actual_concepts:
            actual_concepts.append(word)


outcome_data = data['outcome'].tolist()
actual_outcomes = []
for string in outcome_data:
    words = string.split(';')
    for s in words:
        if s not in actual_outcomes:
            actual_outcomes.append(s)

In [4]:
dsa_2214_chapters = [
    'Data Structures and Algorithms',
    'Mathematical Preliminaries',
    'Algorithm Analysis',
    'Lists, Stacks, and Queues',
    'Binary Trees',
    'Non-Binary Trees',
    'Internal Sorting',
    'File Processing and External Sorting',
    'Searching',
    'Indexing',
    'Graphs',
    'Lists and Arrays Revisited',
    'Advanced Tree Structures',
    'Analysis Techniques',
    'Lower Bounds',
    'Patterns of Algorithms',
    'Limits to Computation',
]

In [5]:
actual_concepts = [' '.join(actual_concepts)] * 4
actual_outcomes = [' '.join(actual_outcomes)] * 4

from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
from ragas.embeddings import LangchainEmbeddingsWrapper
from src.transformerEmbeddings import TransformerEmbeddings

eval_llm = LangchainLLMWrapper(langchain_llm = ChatOpenAI())
eval_embeddings = LangchainEmbeddingsWrapper(embeddings = TransformerEmbeddings())

from ragas.metrics import ResponseRelevancy, LLMContextRecall, SemanticSimilarity, Faithfulness, NoiseSensitivity, LLMContextPrecisionWithReference

In [6]:
metrics = [ResponseRelevancy(), LLMContextPrecisionWithReference(), LLMContextRecall(), SemanticSimilarity(), Faithfulness(llm = eval_llm), NoiseSensitivity()]

<h4>Testing new retrieval class design... (PLEASE WORK)</h4>

In [7]:
from src.extractor import relationExtractor
extractor = relationExtractor(dsa_2214_link, 
                            token, 
                            dsa_2214_chapters[6:10], 
                            'bibliography',
                            getenv('connection_string'),
                            3000,
                            100,
                            'DocumentEmbeddings',
                            '2214_embeddings',
                            True,
                            'gpt-4o-mini',
                            0.1)

In [8]:
retrieved, concepts = extractor.identify_concepts(5)

Concepts evaluation without text normalization

In [10]:
for i in range(5):
    samples = extractor.evaluate('concepts', 5, concepts, actual_concepts, metrics = metrics, data = retrieved)

Evaluating: 100%|██████████| 24/24 [02:56<00:00,  7.35s/it]


{'answer_relevancy': 0.8920, 'llm_context_precision_with_reference': 0.2500, 'context_recall': 0.0980, 'semantic_similarity': 0.8100, 'faithfulness': 0.6582, 'noise_sensitivity_relevant': 0.0661}


Evaluating: 100%|██████████| 24/24 [02:22<00:00,  5.92s/it]


{'answer_relevancy': 0.8920, 'llm_context_precision_with_reference': 0.2500, 'context_recall': 0.0980, 'semantic_similarity': 0.8100, 'faithfulness': 0.7487, 'noise_sensitivity_relevant': 0.1098}


Evaluating: 100%|██████████| 24/24 [02:23<00:00,  5.99s/it]


{'answer_relevancy': 0.8920, 'llm_context_precision_with_reference': 0.2500, 'context_recall': 0.0980, 'semantic_similarity': 0.8100, 'faithfulness': 0.8029, 'noise_sensitivity_relevant': 0.2927}


Evaluating: 100%|██████████| 24/24 [02:28<00:00,  6.19s/it]


{'answer_relevancy': 0.8920, 'llm_context_precision_with_reference': 0.2500, 'context_recall': 0.0980, 'semantic_similarity': 0.8100, 'faithfulness': 0.7875, 'noise_sensitivity_relevant': 0.0066}


Evaluating: 100%|██████████| 24/24 [02:13<00:00,  5.55s/it]

{'answer_relevancy': 0.8916, 'llm_context_precision_with_reference': 0.2500, 'context_recall': 0.0980, 'semantic_similarity': 0.8100, 'faithfulness': 0.9214, 'noise_sensitivity_relevant': 0.0818}


In [15]:
concepts

[['Here are the five most important learning concepts related to the chapter on Internal Sorting, particularly focusing on the stability of sorting algorithms:',
  '1. **Definition of Stability in Sorting Algorithms**:',
  '   - A sorting algorithm is considered stable if it preserves the relative order of records with equal keys. This means that if two elements have the same key, their order in the sorted output will be the same as their order in the input.',
  '2. **Stability of Common Sorting Algorithms**:',
  '   - **Stable Algorithms**:',
  '     - **Insertion Sort**: Stable because it only swaps elements when necessary, preserving the order of equal elements.',
  '     - **Bubble Sort**: Stable as it compares adjacent elements and only swaps them if they are out of order, maintaining the order of equal elements.',
  '     - **Merge Sort**: Stable because it merges sorted subarrays while maintaining the order of equal elements.',
  '     - **Radix Sort**: Stable since it processes

Concepts evaluation with text normalization on retrieved contexts and concepts

In [11]:
normalized_concepts = [[normalize_text(' '.join(t))] for t in concepts]
normalized_truths = [normalize_text(t) for t in actual_concepts]

normalized_retrieved = {}
for k in retrieved.keys():
    normalized_retrieved[k] = normalize_text(retrieved[k])

for i in range(5):
    samples = extractor.evaluate('concepts', 5, normalized_concepts, normalized_truths, data = normalized_retrieved, metrics = metrics)

Evaluating: 100%|██████████| 24/24 [01:59<00:00,  4.98s/it]


{'answer_relevancy': 0.8978, 'llm_context_precision_with_reference': 0.5000, 'context_recall': 0.0500, 'semantic_similarity': 0.8189, 'faithfulness': 0.9348, 'noise_sensitivity_relevant': 0.0192}


Evaluating: 100%|██████████| 24/24 [02:12<00:00,  5.53s/it]


{'answer_relevancy': 0.8984, 'llm_context_precision_with_reference': 0.5000, 'context_recall': 0.0833, 'semantic_similarity': 0.8189, 'faithfulness': 0.8254, 'noise_sensitivity_relevant': 0.0096}


Evaluating: 100%|██████████| 24/24 [02:07<00:00,  5.31s/it]


{'answer_relevancy': 0.8971, 'llm_context_precision_with_reference': 0.5000, 'context_recall': 0.1833, 'semantic_similarity': 0.8189, 'faithfulness': 0.9226, 'noise_sensitivity_relevant': 0.1154}


Evaluating: 100%|██████████| 24/24 [01:55<00:00,  4.83s/it]


{'answer_relevancy': 0.8969, 'llm_context_precision_with_reference': 0.5000, 'context_recall': 0.0833, 'semantic_similarity': 0.8189, 'faithfulness': 0.9194, 'noise_sensitivity_relevant': 0.0096}


Evaluating: 100%|██████████| 24/24 [01:46<00:00,  4.43s/it]


{'answer_relevancy': 0.8966, 'llm_context_precision_with_reference': 0.5000, 'context_recall': 0.0500, 'semantic_similarity': 0.8189, 'faithfulness': 0.9004, 'noise_sensitivity_relevant': 0.0000}


Outcome evaluation using concepts without text normalization

In [12]:
outcomes, outcome_contexts = extractor.identify_outcomes(5)

In [13]:
samples = extractor.evaluate('outcomes', 5, outcomes, actual_outcomes, metrics = metrics, data = retrieved)

Evaluating: 100%|██████████| 24/24 [02:45<00:00,  6.91s/it]

{'answer_relevancy': 0.8746, 'llm_context_precision_with_reference': 0.2500, 'context_recall': 0.0455, 'semantic_similarity': 0.8030, 'faithfulness': 0.6287, 'noise_sensitivity_relevant': 0.1689}


Outcome evaluation with text normalization on outcomes and retrieved contexts

In [14]:
normalized_outcomes = [[normalize_text(' '.join(t))] for t in outcomes]
normalized_truths = [normalize_text(t) for t in actual_outcomes]

normalized_retrieved = {}
for k in retrieved.keys():
    normalized_retrieved[k] = normalize_text(outcome_contexts[k])

samples = extractor.evaluate('outcomes', 5, normalized_outcomes, normalized_truths, data = normalized_retrieved, metrics = metrics)

Evaluating: 100%|██████████| 24/24 [02:00<00:00,  5.04s/it]


{'answer_relevancy': 0.8743, 'llm_context_precision_with_reference': 0.5000, 'context_recall': 0.3750, 'semantic_similarity': 0.8376, 'faithfulness': 0.8313, 'noise_sensitivity_relevant': 0.2539}


In [19]:
chp4_concepts = []
with open('data/chp4.txt', 'r') as f:
    for l in f.readlines():
        concepts = l.strip().split('->')
        for c in concepts:
            if c not in chp4_concepts: chp4_concepts.append(c)

print(chp4_concepts)

['Data Structures', 'Building Blocks', 'ArrayList', 'ArrayList Traversal/Search', 'ArrayList Insertion', 'ArrayList Removing', 'Linked List', 'Reference Variable', 'Singly Linked List', 'SLL Traversal/Search', 'SLL Insertion', 'SLL Removing', 'Doubly Linked List', 'DLL Traversal/Search', 'DLL Insertion', 'DLL Removing', 'Circular Doubly Linked List', 'CDLL Traversal/Search', 'CDLL Insertion', 'CDLL Removing', 'Linear Data Structures', 'Stack', 'Stack Conceptal View', 'Stack Array Implementation', 'Stack Linked List Implementation', 'Queue', 'Queue Conceptal View', 'Queue Array Implementation', 'Queue Linked List Implementation', 'List', 'List Conceptal View', 'List Array Implementation', 'Linked List Implementation']


In [21]:
extractor.build_terminology([[word] for word in chp4_concepts])

[('Data Structures', 'ArrayList'),
 ('Data Structures', 'Linked List'),
 ('Data Structures', 'Singly Linked List'),
 ('Data Structures', 'Doubly Linked List'),
 ('Data Structures', 'Circular Doubly Linked List'),
 ('Data Structures', 'Linear Data Structures'),
 ('Data Structures', 'Stack'),
 ('Data Structures', 'Queue'),
 ('Data Structures', 'List'),
 ('ArrayList', 'Linear Data Structures'),
 ('ArrayList', 'List'),
 ('ArrayList', 'List Conceptal View'),
 ('ArrayList Traversal/Search', 'List'),
 ('ArrayList Insertion', 'Linear Data Structures'),
 ('ArrayList Insertion', 'List'),
 ('ArrayList Removing', 'Linear Data Structures'),
 ('ArrayList Removing', 'List'),
 ('Linked List', 'Singly Linked List'),
 ('Linked List', 'Doubly Linked List'),
 ('Linked List', 'Circular Doubly Linked List'),
 ('Linked List', 'Linear Data Structures'),
 ('Linked List', 'List'),
 ('Linked List', 'List Conceptal View'),
 ('Singly Linked List', 'Data Structures'),
 ('Singly Linked List', 'Linked List'),
 ('Sing